# Probabilistic Graphical Models, SS 2022

In [ ]:
import os
import re
import nltk
import shutil
import gensim
import pprint
import random
import zipfile
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
from gensim.corpora import Dictionary
from sklearn.datasets import load_files
from nltk.corpus import names, stopwords
from gensim.models.phrases import Phraser
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from gensim.models import Phrases, CoherenceModel, LdaModel, HdpModel

stop_words = stopwords.words('english')

## 1. Loading and preprocessing the datasets

### 1.1 Load `20NewsGroup` dataset

In [ ]:
ARCHIVE_NAME_ORIGINAL = "original.zip"
ARCHIVE_NAME_MODIFIED = "modified.zip"

In [ ]:
def load_dataset(archive_name):
    target_dir = 'data/'
    archive_path = os.path.join(target_dir, archive_name)
    if archive_name=="original.zip":        
        path = os.path.join(target_dir, 'original')
    else:
        path = os.path.join(target_dir, 'modified')
       
    print("Decompressing %s" %archive_name)    
    with zipfile.ZipFile(archive_path, 'r') as zip_ref:
        if not os.path.exists(path):
            zip_ref.extractall(target_dir)
        else:
            shutil.rmtree(path)
            zip_ref.extractall(target_dir) 
    
    # Load text files with categories as subfolder names
    # Decode text files, if encoding=None load_files() returns list of bytes
    dataset = load_files(path, load_content=True, encoding='latin1')
    data = dataset.data
    print("Data is decompressed!") 
    
    return data

#### Load '20NewsGroup' dataset in two formats: the original (orig), and a slightly modified (mod) version

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

dataset_original = load_dataset(ARCHIVE_NAME_ORIGINAL)
print('Number of original newsgroup documents: %d\n' % len(dataset_original))

dataset_modified = load_dataset(ARCHIVE_NAME_MODIFIED)
print('Number of modified newsgroup documents: %d' % len(dataset_modified))

### 1.2 Preprocess the datasets

In [ ]:
#remove the documents' header
#def remove_text_header(text):
 #   header, blankline, content = text.partition('\n\n')
  #  return content

#remove the documents' footer/signature
#def remove_text_footer(text):
 #   text = text.replace('-', '')        
  #  content, blankline, footer = text.rpartition('\n\n')
   # return content

In [ ]:
def preprocess(dataset):  
    min_tokens = 25
    
    # remove document with fewer than 25 tokens
    dataset = [paper for paper in dataset if len(paper.split())>min_tokens]
    
    # remove emails
    dataset = [re.sub('\S*@\S*\s?', '', paper) for paper in dataset] 
        
    # convert to lowercase and split document into tokens
    for i in tqdm(range(len(dataset))):
        # remove punctuation, numbers and tokens that contain numbers
        tokenizer = RegexpTokenizer(r'[a-zA-Z]{3,}') # r'\w+'
        dataset[i] = dataset[i].lower()
        dataset[i] = tokenizer.tokenize(dataset[i]) 
    
    #remove numbers but not words that contains numbers
    #data = [[token for token in paper if not token.isnumeric()] for paper in dataset]

    # remove tokens with less than three-character string
    #dataset = [[token for token in paper if len(token)>2] for paper in dataset]
    
    # remove stop words
    stop_words.extend(['from','subject','re','edu','use','cmu'])
    dataset = [[token for token in paper if token not in stop_words] for paper in dataset]
    
    # add bigrams to docs (only ones that appear 20 times or more, 
    # with the score of the phrase greater than threshold)
    bigram = Phrases(dataset, min_count=20, scoring='npmi', threshold=0.8)
    bigram_mod = Phraser(bigram)
    dataset = [bigram_mod[paper] for paper in dataset]
    
    #map pos_tag to first character lemmatize() accepts
    #tags = lambda e: ('a' if e[0].lower() == 'j' else e[0].lower()) if e[0].lower() in ['n', 'r', 'v'] else 'n'
    #tags(nltk.pos_tag([token])[0][1][0])

    lemmatizer = WordNetLemmatizer()
    
    # lemmatize with corresponding pos_tag: NOUN='n', VERB='v'
    dataset = [[lemmatizer.lemmatize(token, 'v') for token in paper] for paper in dataset]
    dataset = [[lemmatizer.lemmatize(token, 'n') for token in paper] for paper in dataset]
        
    # create vocabulary
    dictionary = Dictionary(dataset)

    # filter out words that occur less than 50 documents, or more than 80% of the documents.
    dictionary.filter_extremes(no_below=50, no_above=0.8)
    corpus = [dictionary.doc2bow(doc) for doc in dataset]

    print('Number of unique tokens: %d' % len(dictionary))
    print('Number of documents - corpus size: %d' % len(corpus))
    
        
    return dataset, corpus, dictionary   

#### Preprocess '20NewsGroup' dataset in two formats: the original (orig), and a slightly modified (mod) version

In [ ]:
papers_orig, corpus_orig, dictionary_orig = preprocess(dataset_original)
print('Number of original newsgroup documents: %d\n' % len(papers_orig))
    
papers_mod, corpus_mod, dictionary_mod = preprocess(dataset_modified)
print('Number of modified newsgroup documents: %d' % len(papers_mod))

## 2. Latent Dirichlet Allocation (LDA, parametric) topic model using `gensim`

#### Set training parameters and run the LDA training

In [ ]:
def tm1(data, k, corpus, dictionary):
    
    # Set training parameters.
    num_topics = k
    chunksize = 100
    passes = 20
    iterations = 1000
    random_state = 0
    alpha='auto'
    eta='auto'
    eval_every = None  # Don't evaluate model perplexity, takes too much time.

    # Make an index to word dictionary.
    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token
    
    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha=alpha,
        eta=eta,
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        random_state=random_state,
        eval_every=eval_every
    )
    
    return model

## 3. Hierarchical Dirichlet Process (HDP, non-parametric) topic model using `gensim`

#### Set training parameters and run the HDP training

In [ ]:
def tm2(corpus, dictionary):
    
    # Set training parameters.
    chunksize = 100
    K = 10 # prior - initial number of topics
    T = 50 # max number of topics
    alpha=1 
    gamma=1 
    eta=0.01

    # Make an index to word dictionary.
    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token
    
    model = HdpModel(
        corpus=corpus,
        id2word=id2word,
        chunksize = chunksize,
        K=K,
        T=T,
        alpha=alpha, 
        gamma=gamma, 
        eta=eta
    )
    
    return model

## 4. Training LDA and HDP models on both versions of the dataset

### 4.1 LDA Training for `papers_orig`

In [ ]:
tm1_orig = tm1(papers_orig, 20, corpus_orig, dictionary_orig)

ldatopics_orig = [[term for term, wt in tm1_orig.show_topic(n, topn=10)] for n in range(0, tm1_orig.num_topics)]
for i, topic in enumerate(ldatopics_orig):
        print(i, "-", topic)

### 4.2 LDA Training for `papers_mod`

In [ ]:
tm1_mod = tm1(papers_mod, 20, corpus_mod, dictionary_mod)

ldatopics_mod = [[term for term, wt in tm1_mod.show_topic(n, topn=10)] for n in range(0, tm1_mod.num_topics)]
for i, topic in enumerate(ldatopics_mod):
        print(i, "-", topic)

### 4.3 HDP Training for `papers_orig`

In [ ]:
tm2_orig = tm2(corpus_orig, dictionary_orig)
hdptopics_orig = [[term for term, wt in tm2_orig.show_topic(n, topn=10)] for n in range(tm2_orig.m_T)]

for i, topic in enumerate(hdptopics_orig[:20]):
        print(i, "-", topic)

### 4.4 HDP Training for `papers_mod`

In [ ]:
tm2_mod = tm2(corpus_mod, dictionary_mod)
hdptopics_mod = [[term for term, wt in tm2_mod.show_topic(n, topn=10)] for n in range(tm2_mod.m_T)]

for i, topic in enumerate(hdptopics_mod[:20]):
        print(i, "-", topic)

## 5. Perplexity and coherence scores analysis

### 5.1 LDA Perplexity and Coherence for all the models 

In [ ]:
def calculate_metrics_lda(papers, corpus, dictionary, model):
    perp_score = model.log_perplexity(corpus)
    
    coherence_model = CoherenceModel(
        model=model, 
        texts=papers, 
        dictionary=dictionary, 
        coherence='c_v'
    ) 
    
    coherence_score = coherence_model.get_coherence()
    
    return perp_score, coherence_score

In [ ]:
def evaluate_bar_graph(coherences, models):
    """
    Function to plot bar graph.
    
    coherences: list of coherence values
    models: model names to mark bars.
    """
    n = len(coherences)
    x = np.arange(n)
    plt.bar(x, coherences, width=0.5, tick_label=models, align='center')
    plt.xlabel('Models')
    plt.ylabel('Coherence Value')

In [ ]:
perp_lda_orig, coher_lda_orig = calculate_metrics_lda(papers_orig, corpus_orig, dictionary_orig, tm1_orig)
print("LDA_orig: Perplexity = {}, Coherence = {} \n".format(str(perp_lda_orig), str(coher_lda_orig)))
perp_lda_mod, coher_lda_mod = calculate_metrics_lda(papers_mod, corpus_mod, dictionary_mod, tm1_mod)
print("LDA_mod: Perplexity = {}, Coherence = {} \n".format(str(perp_lda_mod), str(coher_lda_mod)))

In [ ]:
coher_hdp_orig = CoherenceModel(
    topics=hdptopics_orig[:20], 
    texts=papers_orig, 
    dictionary=dictionary_orig).get_coherence()

print("HDP_orig: Coherence = {} \n".format(str(coher_hdp_orig)))

coher_hdp_mod = CoherenceModel(
    topics=hdptopics_mod[:20], 
    texts=papers_mod, 
    dictionary=dictionary_mod).get_coherence()

print("HDP_mod: Coherence = {} \n".format(str(coher_hdp_mod)))

In [ ]:
coherences = [coher_lda_orig, coher_lda_mod, coher_hdp_orig, coher_hdp_mod]
model_names = ['LDA_orig', 'LDA_mod', 'HDP_orig', 'HDP_mod']

evaluate_bar_graph(coherences, model_names)

### 5.2 What are the differences in topics for the two dataset versions?

### 5.3 Effect of data size and number of topics on perplexity and coherence scores

In [ ]:
def effectOfDataAndK_LDA(datasets, datasetNames, K):
    print("K: ", K)
    print("total Tests: {} datasets * {} values of k = {}".format(str(len(datasets)), str(len(K)), 
                                                                  str(len(datasets) * len(K))))
    perplexityScores = {} 
    coherenceScores = {}
    for datasetName, papers in zip(datasetNames, datasets):
        print("running for dataset ", datasetName)
        data_prep, corpus, dictionary = preprocess(papers)
        
        perplexityForK = {}
        coherenceForK = {}
        for k in K:
            model = tm1(data_prep, k, corpus, dictionary)
            perp_score, coherence_lda = calculate_metrics_lda(data_prep, corpus, dictionary, model)
            
            print("For {} and k = {}: perplexity = {}, coherence = {}".format(datasetName, str(k), 
                                                                              str(perp_score), str(coherence_lda)))
            perplexityForK[str(k)] = perp_score
            coherenceForK[str(k)] = coherence_lda
            
        perplexityScores[datasetName] = perplexityForK
        coherenceScores[datasetName] = coherenceForK
        
    return perplexityScores, coherenceScores

In [ ]:
def plot_perplexity(score, data_name):
    
    #plotting perplexity
    %matplotlib inline

    fig, axes = plt.subplots(3, 1, figsize=(15, 10))
    K = [i*3 for i in range(1, 11)]
    fig.tight_layout()

    #graph when N is same for all k
    for i in range(len(data_name)):
        perp_scoreDict = score[data_name[i]]
        perp_scores = []
        for numTopics in K:
            perp_scores.append(perp_scoreDict[str(numTopics)])
        
        axes[i].set_xticks(K)
        axes[i].plot(K, perp_scores)
        axes[i].set_xlabel("number of topics (k)")
        axes[i].set_ylabel("log_Perplexity")
        axes[i].set_title(data_name[i])

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_coherence(score, data_name):
    
    #plotting perplexity
    %matplotlib inline

    fig, axes = plt.subplots(3, 1, figsize=(15, 10))
    K = [i*3 for i in range(1, 11)]
    fig.tight_layout()

    #graph when N is same for all k
    for i in range(len(data_name)):
        coher_scoreDict = score[data_name[i]]
        coher_scores = []
        for numTopics in K:
            coher_scores.append(coher_scoreDict[str(numTopics)])
        
        axes[i].set_xticks(K)
        axes[i].plot(K, coher_scores)
        axes[i].set_xlabel("number of topics (k)")
        axes[i].set_ylabel("Coherence")
        axes[i].set_title(data_name[i])

    plt.tight_layout()
    plt.show()

#### 5.3.1 Making three datasets with 11998 (60%), 15998 (80%), and 19997(100%) documents

###### Randomly sampling papers

In [ ]:
def sample(data, numSamples):
    randomSamples = random.sample(data, numSamples)
    return randomSamples

##### 5.3.1.1 LDA for `papers_orig`

In [ ]:
data_original = load_dataset(ARCHIVE_NAME_ORIGINAL)

#sample the data
papers_orig_60 = sample(data_original, 11998)
papers_orig_80 = sample(data_original, 15998)
papers_orig_100 = data_original

#### Checking the effect of change in 'k' and 'N'

In [ ]:
k = [i*3 for i in range(1, 11)]
datasets_paper_orig = [papers_orig_60, papers_orig_80, papers_orig_100]
datasetNames_orig = ["papers_orig_60", "papers_orig_80", "papers_orig_100"]
ldaperp_orig, ldacoher_orig = effectOfDataAndK_LDA(datasets_paper_orig, datasetNames_orig, k)

In [ ]:
plot_perplexity(ldaperp_orig, datasetNames_orig)

In [ ]:
plot_coherence(ldacoher_orig, datasetNames_orig)

##### 5.3.1.2 LDA for `papers_mod`

In [ ]:
data_modified = load_dataset(ARCHIVE_NAME_MODIFIED)

#sample the data
papers_mod_60 = sample(data_modified, 11998)
papers_mod_80 = sample(data_modified, 15998)
papers_mod_100 = data_modified

In [ ]:
k = [i*3 for i in range(1, 11)]
datasets_paper_mod = [papers_mod_60, papers_mod_80, papers_mod_100]
datasetNames_mod = ["papers_mod_60", "papers_mod_80", "papers_mod_100"]
ldaperp_mod, ldacoher_mod = effectOfDataAndK_LDA(datasets_paper_mod, datasetNames_mod, k)

In [ ]:
plot_perplexity(perpScores_mod, datasetNames_mod)

In [ ]:
plot_coherence(coherScores_mod, datasetNames_mod)

##### 5.3.1.3 HDP for `papers_orig`

In [ ]:
def effectOfDataAndK_HDP(datasets, datasetNames, K):
    print("K: ", K)
    print("total Tests: {} datasets * {} values of k = {}".format(str(len(datasets)), str(len(K)), 
                                                                  str(len(datasets) * len(K))))
    coherenceScores = {}
    for datasetName, papers in zip(datasetNames, datasets):
        print("running for dataset ", datasetName)
        data_prep, corpus, dictionary = preprocess(papers)
        
        model = tm2(corpus, dictionary)
        hdptopics = [[term for term, wt in model.show_topic(n, topn=10)] for n in range(model.m_T)]
        
        coherenceForK = {}
        for k in K:            
            coherence_hdp = CoherenceModel(
                topics=hdptopics[:k], 
                texts=data_prep, 
                dictionary=dictionary).get_coherence()
            
            print("For {} and k = {}: coherence = {}".format(datasetName, str(k), str(coherence_hdp)))
            coherenceForK[str(k)] = coherence_hdp
            
        coherenceScores[datasetName] = coherenceForK
        
    return coherenceScores

In [ ]:
k = [i*3 for i in range(1, 11)]
datasets_paper_orig = [papers_orig_60, papers_orig_80, papers_orig_100]
datasetNames_orig = ["papers_orig_60", "papers_orig_80", "papers_orig_100"]
hdpcoher_orig = effectOfDataAndK_HDP(datasets_paper_orig, datasetNames_orig, k)

In [ ]:
plot_coherence(hdpcoher_orig, datasetNames_orig)

##### 5.3.1.4 HDP for `papers_mod`

In [ ]:
k = [i*3 for i in range(1, 11)]
datasets_paper_mod = [papers_mod_60, papers_mod_80, papers_mod_100]
datasetNames_mod = ["papers_mod_60", "papers_mod_80", "papers_mod_100"]
hdpcoher_mod = effectOfDataAndK_HDP(datasets_paper_mod, datasetNames_mod, k)

In [ ]:
plot_coherence(hdpcoher_mod, datasetNames_mod)

#### 5.3.2 Making three datasets with 500 000, 1 500 000, 2 500 000 sentences

In [ ]:
def getSentenceBasedCorpus(data, N):
    # M = 19997 for data_original
    # M = 18397 for data_modified
    #M samples will have N/M number of sentences from each sample
    #get sentences from the data
    
    sentences = [sent_tokenize(paper) for paper in data]
    n = int(N/len(sentences))
    sentenceData = ["".join(i[:n]) for i in sentences if len(i) != 0]
    
    return sentenceData

##### 5.3.2.1 LDA for `papers_orig`

In [ ]:
nltk.download('punkt')

sentences_500000_orig = getSentenceBasedCorpus(data_original, 500000)
sentences_1500000_orig = getSentenceBasedCorpus(data_original, 1500000)
sentences_2500000_orig = getSentenceBasedCorpus(data_original, 2500000)

In [ ]:
k = [i*3 for i in range(1, 11)]
datasets_sentence_orig = [sentences_500000_orig, sentences_1500000_orig, sentences_2500000_orig]
datasetNames_sent_orig = ["sent_orig_500000", "sent_orig_1500000", "sent_orig_2500000"]
ldaperp_sent_orig, ldacoher_sent_orig = effectOfDataAndK(datasets_sentence_orig, datasetNames_sent_orig, k)

In [ ]:
plot_perplexity(ldaperp_sent_orig, datasetNames_sent_orig)

In [ ]:
plot_coherence(ldacoher_sent_orig, datasetNames_sent_orig)

##### 5.3.2.2 LDA for `papers_mod`

In [ ]:
nltk.download('punkt')

sentences_500000_mod = getSentenceBasedCorpus(data_modified, 500000)
sentences_1500000_mod = getSentenceBasedCorpus(data_modified, 1500000)
sentences_2500000_mod = getSentenceBasedCorpus(data_modified, 2500000)

In [ ]:
k = [i*3 for i in range(1, 11)]
datasets_sentence_mod = [sentences_500000_mod, sentences_1500000_mod, sentences_2500000_mod]
datasetNames_sent_mod = ["sent_mod_500000", "sent_mod_1500000", "sent_mod_2500000"]
ldaperp_sent_mod, ldacoher_sent_mod = effectOfDataAndK(datasets_sentence_mod, datasetNames_sent_mod, k)

In [ ]:
plot_perplexity(ldaperp_sent_mod, datasetNames_sent_mod)

In [ ]:
plot_coherence(ldacoher_sent_mod, datasetNames_sent_mod)

##### 5.3.2.3 HDP for `papers_orig`

In [ ]:
k = [i*3 for i in range(1, 11)]
datasets_sentence_orig = [sentences_500000_orig, sentences_1500000_orig, sentences_2500000_orig]
datasetNames_sent_orig = ["sent_orig_500000", "sent_orig_1500000", "sent_orig_2500000"]
hdpcoher_sent_orig = effectOfDataAndK_HDP(datasets_sentence_orig, datasetNames_sent_orig, k)

In [ ]:
plot_coherence(hdpcoher_sent_orig, datasetNames_sent_orig)

##### 5.3.2.4 HDP for `papers_mod`

In [ ]:
k = [i*3 for i in range(1, 11)]
datasets_sentence_mod = [sentences_500000_mod, sentences_1500000_mod, sentences_2500000_mod]
datasetNames_sent_mod = ["sent_mod_500000", "sent_mod_1500000", "sent_mod_2500000"]
hdpcoher_sent_mod = effectOfDataAndK_HDP(datasets_sentence_mod, datasetNames_sent_mod, k)

In [ ]:
plot_coherence(hdpcoher_sent_mod, datasetNames_sent_mod)

## 6. Further improvements

In [ ]:
data_prep, corpus, dictionary = preprocess(papers)

# Set training parameters.
num_topics = 20
chunksize = 100
passes = 20
iterations = 1000
random_state = 0
alpha=[0.01, 0.05, 0.1, 0.25, 1.0, 5.0]
eta=[0.01, 0.05, 0.1]
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

for a in alpha:
    for b in eta:
        model = LdaModel(
            corpus=corpus,
            id2word=id2word,
            chunksize=chunksize,
            alpha=a,
            eta=b,
            iterations=iterations,
            num_topics=num_topics,
            passes=passes,
            random_state=random_state,
            eval_every=eval_every
        )
        

In [ ]:
alpha = [0.01, 0.05, 0.1, 0.25, 1.0, 5.0]
eta = [0.01, 0.05, 0.1]